In [21]:
# Standard library imports
from datetime import datetime
import time
import math
import warnings

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import MetaTrader5 as mt
import pmdarima as pm
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tsa.arima.model import ARIMA



In [22]:
mt.initialize()

False

In [23]:
#This is a demo account I have created to sign into when on Metatrader5
login = 10001610777
password = 'Td@oQj5f'
server = 'MetaQuotes-Demo'
mt.login(login, password, server)

False

In [24]:
#MT5 DATASET
symbol = 'XAUUSD'
timeframe = mt.TIMEFRAME_D1
date_from = datetime(2023, 2, 15)
date_to = datetime(2024, 2, 5)

mt_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, date_from, date_to))
mt_data['time'] = pd.to_datetime(mt_data['time'], unit='s')
mt_data.set_index('time', inplace=True)


mt_data


KeyError: 'time'

In [ ]:
# Plot MT5 data
plt.plot(mt_data.index, mt_data['close'])
plt.xlabel('Date')
plt.ylabel('Closing Prices')
plt.title('MT5 Closing Prices')
plt.show()

KeyError: 'close'

In [ ]:
#Train Test Split

to_row = int(len(mt_data) * 0.9)
training_data = list(mt_data[0:to_row]['close'])
testing_data = list(mt_data[to_row:]['close'])
testing_data


In [ ]:
# Split data into test and training set
plt.figure(figsize=(10, 6))
plt.grid(True)
plt.plot(mt_data.index[:to_row], training_data, color='green', label='Train data')
plt.plot(mt_data.index[to_row:], testing_data, color='blue', label='Test data')
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.title('Train-Test Split')
plt.legend()
plt.show()


In [ ]:
warnings.simplefilter('ignore', category=ConvergenceWarning)


In [ ]:
model_predictions = []
n_test_obser = len(testing_data)


In [ ]:
autoarima_model = pm.auto_arima(training_data, seasonal=False, suppress_warnings=True)



In [ ]:
best_pdq = autoarima_model.order

In [ ]:
for i in range(n_test_obser):
    model = sm.tsa.ARIMA(training_data, order=best_pdq)
    model_fit = model.fit()
    output = model_fit.forecast(steps=1)
    yhat = output[0]
    model_predictions.append(yhat)
    actual_test_value = testing_data[i]
    training_data.append(actual_test_value)


In [ ]:
print(model_fit.summary())

In [ ]:
print("Final Prediction:", model_predictions[-1])

In [ ]:

plt.grid(True)
date_range = mt_data.index[to_row:to_row + len(testing_data)]

# Plot the predicted and actual prices
plt.plot(date_range, model_predictions[:len(date_range)], color='blue', marker='o', linestyle='dashed', label='XAUUSD Predicted Price')
plt.plot(date_range, testing_data, color='red', marker='o', label='XAUUSD Actual Price')

plt.title('XAUUSD Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
#report performance
mape = np.mean(np.abs(np.array(model_predictions[:len(testing_data)]) - np.array(testing_data)) / np.abs(testing_data))
print('MAPE: ' + str(mape))




In [ ]:
# Prepare data for rendering in the template
forex_symbol = symbol
current_price = mt_data['close'].iloc[-1]  # Get the last closing price
time_interval = '1 day'  # Update this based on your data
prediction_models = ['ARIMA', 'AutoARIMA']  # Update this based on the models used

    # Create context for rendering the template
context = {
        'forex_symbol': forex_symbol,
        'current_price': current_price,
        'time_interval': time_interval,
        'prediction_models': prediction_models,
        'model_predictions': model_predictions,
        'testing_data': testing_data,
        'mape': mape
    }